In [47]:
import pandas as pd
df = pd.read_csv("Ad click prediction/Raw data/train.csv",nrows = 100000)
df.fillna("Unknown",inplace = True)
df.siteid = df.siteid.astype(str,errors = "ignore")

In [48]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in ['offerid','category','merchant','countrycode','browserid','devid',"siteid"]:
    df[i] = le.fit_transform(df[i])

In [11]:
#features_train = df[['siteid','offerid','category','merchant','countrycode',
#                     'browserid','devid']]
features_train = df[['offerid','category','merchant','countrycode',
                     'browserid','devid']]
target = df[['click']]

In [12]:
features_train.head()

,offerid,category,merchant,countrycode,browserid,devid
0,66686,48,116,4,2,3
1,13517,57,58,1,8,0
2,39037,67,13,0,1,3
3,29515,114,456,2,2,1
4,35611,36,246,3,8,0


In [13]:
# Utility function to report best scores
from time import strftime
def report(results, n_top=5):
    print("Report generated at {}".format(strftime("%d-%m-%Y %H:%M:%S")))
    print("")
    
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.6f} (std: {1:.6f}),time:{2:.3f}".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate],
                results["mean_fit_time"][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [15]:
from time import time
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.externals import joblib
from time import strftime
from sklearn import model_selection
import numpy as np


# build a classifier
clf = XGBClassifier()


# specify parameters and distributions to sample from
param_dist = {#'min_samples_split':sp_randint(60,400),
              "learning_rate":sp_uniform(0.01,0.4),
              #"gamma":sp_uniform(0.001,30),
              "n_estimators":sp_randint(10,100),
              "max_depth":sp_randint(3,12),
              "colsample_bytree":np.random.uniform(0.3,0.99,1000),
               'subsample': np.random.uniform(0.3,0.999,1000),
               'reg_lambda': sp_uniform(0.3,2)}

# run randomized search
#kfold = model_selection.KFold(n_splits=3, random_state=10)

n_iter_search = 10
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,scoring='roc_auc',cv=3
                                   ,n_jobs=-1)



start = time()
random_search.fit(features_train, target.values.ravel())
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

# activate this when fitting models

best_score =  random_search.best_score_
current_time = strftime("%d-%b-%Y %H-%M-%S")
joblib.dump(random_search, 'Ad click prediction/classifier pickle dump/adclick_{0}'
                           'pv score {1:.6f}.pkl'.format(current_time,best_score))
report(random_search.cv_results_)

RandomizedSearchCV took 64.63 seconds for 10 candidates parameter settings.
Report generated at 02-08-2017 17:06:28

Model with rank: 1
Mean validation score: 0.963944 (std: 0.001207),time:7.326
Parameters: {'subsample': 0.3242467767866315, 'learning_rate': 0.089963993047809415, 'colsample_bytree': 0.47155492779349761, 'reg_lambda': 1.7036775511523934, 'n_estimators': 78, 'max_depth': 4}

Model with rank: 2
Mean validation score: 0.963714 (std: 0.001666),time:5.333
Parameters: {'subsample': 0.62130394398893785, 'learning_rate': 0.21149544049716495, 'colsample_bytree': 0.38842531433733063, 'reg_lambda': 1.7911939273670419, 'n_estimators': 40, 'max_depth': 11}

Model with rank: 3
Mean validation score: 0.963304 (std: 0.001149),time:4.888
Parameters: {'subsample': 0.60541249317136736, 'learning_rate': 0.23852779621799558, 'colsample_bytree': 0.94601906797915758, 'reg_lambda': 0.33558349093614875, 'n_estimators': 62, 'max_depth': 3}

Model with rank: 4
Mean validation score: 0.962915 (std:

In [16]:
report(random_search.cv_results_)

Report generated at 02-08-2017 17:06:48

Model with rank: 1
Mean validation score: 0.963944 (std: 0.001207),time:7.326
Parameters: {'subsample': 0.3242467767866315, 'learning_rate': 0.089963993047809415, 'colsample_bytree': 0.47155492779349761, 'reg_lambda': 1.7036775511523934, 'n_estimators': 78, 'max_depth': 4}

Model with rank: 2
Mean validation score: 0.963714 (std: 0.001666),time:5.333
Parameters: {'subsample': 0.62130394398893785, 'learning_rate': 0.21149544049716495, 'colsample_bytree': 0.38842531433733063, 'reg_lambda': 1.7911939273670419, 'n_estimators': 40, 'max_depth': 11}

Model with rank: 3
Mean validation score: 0.963304 (std: 0.001149),time:4.888
Parameters: {'subsample': 0.60541249317136736, 'learning_rate': 0.23852779621799558, 'colsample_bytree': 0.94601906797915758, 'reg_lambda': 0.33558349093614875, 'n_estimators': 62, 'max_depth': 3}

Model with rank: 4
Mean validation score: 0.962915 (std: 0.001169),time:5.492
Parameters: {'subsample': 0.55734653544257462, 'learni

# Now recover the pickle file and run a test

In [1]:
#temporary
from sklearn.externals import joblib
clf = joblib.load('Ad click prediction/classifier pickle dump/adclick_26-Jul'
                  '-2017 16-37-11 pv score 0.964318.pkl')

C:\Users\user\Anaconda3.6\envs\python35env\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [34]:
le.classes_

array(['1000037.0', '1000107.0', '1000146.0', ..., '9999979.0',
       '9999996.0', 'Unknown'], dtype=object)

In [45]:
import pandas as pd
df_test = pd.read_csv("Ad click prediction/Raw data/test.csv",nrows = 100000)
print("loaded")
df_test.fillna("Unknown",inplace = True)
df_test.siteid = df_test.siteid.astype(str,errors = "ignore")

loaded


In [41]:
df_test.offerid.describe()

count    100000.000000
mean     487240.213560
std      294406.986255
min           2.000000
25%      231515.500000
50%      473943.500000
75%      745273.750000
max      999995.000000
Name: offerid, dtype: float64

In [51]:
for i in ['merchant','countrycode','browserid','devid','offerid','category']:

    df_test[i] = le.transform(df_test[i])
    print(i, "has been transformed")

TypeError: unorderable types: str() > int()